In [19]:
# Stdlib imports
from io import BytesIO

# Third-party libraries
from IPython.display import Image
from ipywidgets import interact, interactive, fixed

import matplotlib as mpl
from skimage import data, filters, io, img_as_uint
import numpy as np

import skimage as sk
from skimage import filters
import os
from scipy import ndimage
import scipy
import sys
# The plotting module matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# The image processing package scipy.ndimage as ndi
import scipy.ndimage as ndi

# %matplotlib inline
# filename = r'181220_Hyb1_pos1_4umSteps-0005.tif'
dirpath = r'test/pics/pics181220/raw'
# filepath = os.path.join(dirpath, filename)
# print(filepath)
#from dcde.pre_annotation.contrast_adjustment import contrast

#import ipywidgets as widgets
from ipywidgets import interact
from skimage.io import imread
# img = imread(filepath)



In [20]:
def arr2img(arr):
    """Display a 2- or 3-d numpy array as an image."""
    if arr.ndim == 2:
        format, cmap = 'png', mpl.cm.gray
    elif arr.ndim == 3:
        format, cmap = 'jpg', None
    else:
        raise ValueError("Only 2- or 3-d arrays can be displayed as images.")
    # Don't let matplotlib autoscale the color range so we can control overall luminosity
    vmax = 255 if arr.dtype == 'uint8' else 1.0
    with BytesIO() as buffer:
        mpl.image.imsave(buffer, arr, format=format, cmap=cmap)
        out = buffer.getvalue()
    return Image(out)


In [21]:
def choose_img(name):
    # Let's store the result in the global `img` that we can then use in our image editor below
    global img
    filepath = os.path.join(dirpath, name)
    img = img_as_uint(imread(filepath))
    return arr2img(img)

interact(choose_img, name=sorted(set(os.listdir(dirpath))));
    

interactive(children=(Dropdown(description='name', options=('181220_Hyb1_pos1_4umSteps-0001.tif', '181220_Hyb1…

In [24]:
def edit_image(image, sigma=1.0, equalize_hist=False, equalize_adapthist=False):
    
    global hist
    global adapthist
    global gaussian_sigma
    
    new_image = filters.gaussian(image, sigma=sigma, multichannel=False)
    new_image += 1000*sk.filters.sobel(new_image)
    new_image[:] = -1.0*new_image[:]
    new_image=sk.exposure.rescale_intensity(new_image, in_range = 'image', out_range = 'float')
    
    if(equalize_hist == True):
        #new_image=sk.exposure.rescale_intensity(new_image, in_range = 'image', out_range = 'np.uint16')
        new_image = sk.exposure.equalize_hist(new_image, nbins=256, mask=None)
        
    if(equalize_adapthist == True):
        new_image = sk.exposure.equalize_adapthist(new_image, kernel_size=None, clip_limit=0.01, nbins=256)
     
    
    hist = equalize_hist
    adapthist = equalize_adapthist
    gaussian_sigma = sigma
        
    
    return arr2img(new_image)
    

In [25]:

interact(edit_image, image=fixed(img), sigma=(0.0,4,0.3));

interactive(children=(FloatSlider(value=1.0, description='sigma', max=4.0, step=0.3), Checkbox(value=False, de…